<a href="https://colab.research.google.com/github/oprostep/2023-fall_LP/blob/main/231126_Hotel_Reservation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np

import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

from keras.callbacks import EarlyStopping

In [2]:
Url="https://raw.githubusercontent.com/oprostep/2023-fall_LP/main/Hotel%20Reservations.csv"
df = pd.read_csv(Url)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

In [4]:
df.isnull().sum()

Booking_ID                              0
no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_year                            0
arrival_month                           0
arrival_date                            0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
booking_status                          0
dtype: int64

In [5]:
new_df = df.drop('Booking_ID',axis=1)
new_df

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled
36271,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
36272,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled
36273,2,0,0,3,Not Selected,0,Room_Type 1,63,2018,4,21,Online,0,0,0,94.50,0,Canceled


In [6]:
X_ = new_df.drop(['type_of_meal_plan','room_type_reserved','market_segment_type','booking_status'],axis=1).astype('float64')

In [7]:
ohe = OneHotEncoder()
Type_of_meal_plan = ohe.fit_transform(new_df.iloc[:,4:5]).toarray()

Type_of_meal_plan

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

In [8]:
new_df['type_of_meal_plan'].value_counts()

Meal Plan 1     27835
Not Selected     5130
Meal Plan 2      3305
Meal Plan 3         5
Name: type_of_meal_plan, dtype: int64

In [9]:
Type_Meal = pd.DataFrame(data = Type_of_meal_plan,index=range(36275),columns=['Meal Plan 1','Not Selected','Meal Plan 2','Meal Plan 3'])
Type_Meal

,Meal Plan 1,Not Selected,Meal Plan 2,Meal Plan 3
0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0
...,...,...,...,...
36270,1.0,0.0,0.0,0.0
36271,1.0,0.0,0.0,0.0
36272,1.0,0.0,0.0,0.0
36273,0.0,0.0,0.0,1.0


In [10]:
new_df['market_segment_type'].value_counts()

Online           23214
Offline          10528
Corporate         2017
Complementary      391
Aviation           125
Name: market_segment_type, dtype: int64

In [11]:
Market_segmet_type = ohe.fit_transform(new_df.iloc[:,11:12]).toarray()
Market_segmet = pd.DataFrame(Market_segmet_type,index=range(36275),columns=['Online','Offline','Corporate','Complementary','Aviation'])
Market_segmet

,Online,Offline,Corporate,Complementary,Aviation
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
36270,0.0,0.0,0.0,0.0,1.0
36271,0.0,0.0,0.0,0.0,1.0
36272,0.0,0.0,0.0,0.0,1.0
36273,0.0,0.0,0.0,0.0,1.0


In [12]:
new_df['room_type_reserved'].value_counts()

Room_Type 1    28130
Room_Type 4     6057
Room_Type 6      966
Room_Type 2      692
Room_Type 5      265
Room_Type 7      158
Room_Type 3        7
Name: room_type_reserved, dtype: int64

In [13]:
Room_type_reserved = ohe.fit_transform(new_df.iloc[:,6:7]).toarray()
Room_type = pd.DataFrame(Room_type_reserved,index=range(36275),columns=['Room_Type 1','Room_Type 2','Room_Type 3','Room_Type 4','Room_Type 5','Room_Type 6','Room_Type 7'])
Room_type

,Room_Type 1,Room_Type 2,Room_Type 3,Room_Type 4,Room_Type 5,Room_Type 6,Room_Type 7
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
36270,0.0,0.0,0.0,1.0,0.0,0.0,0.0
36271,1.0,0.0,0.0,0.0,0.0,0.0,0.0
36272,1.0,0.0,0.0,0.0,0.0,0.0,0.0
36273,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
t = pd.concat([Type_Meal,Market_segmet],axis=1)

In [15]:
t2 = pd.concat([t,Room_type],axis=1)

In [16]:
X = pd.concat([X_,t2],axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  float64
 1   no_of_children                        36275 non-null  float64
 2   no_of_weekend_nights                  36275 non-null  float64
 3   no_of_week_nights                     36275 non-null  float64
 4   required_car_parking_space            36275 non-null  float64
 5   lead_time                             36275 non-null  float64
 6   arrival_year                          36275 non-null  float64
 7   arrival_month                         36275 non-null  float64
 8   arrival_date                          36275 non-null  float64
 9   repeated_guest                        36275 non-null  float64
 10  no_of_previous_cancellations          36275 non-null  float64
 11  no_of_previous_

In [17]:
le = LabelEncoder()
y = le.fit_transform(new_df['booking_status'])

In [23]:
k=5

kfold = KFold(n_splits=k, shuffle=True)

acc_score = []

def model_fn():
  model = Sequential()
  model.add(Dense(24, input_dim=30, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  return model

for train_index, test_index in kfold.split(X):
  X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
  y_train, y_test = y[train_index], y[test_index]

  model = model_fn()
  model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
  early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
  history = model.fit(X_train, y_train, epochs = 10, batch_size = 10, validation_data = (X_test, y_test), callbacks=[early_stop])

  accuracy = model.evaluate(X_test, y_test)[1]
  acc_score.append(accuracy)

avg_acc_score = sum(acc_score)

print('정확도: ', acc_score)
print('정확도 평균: ', avg_acc_score)


Epoch 1/100
2902/2902 [==============================] - 5s 1ms/step - loss: 1.5266 - accuracy: 0.7145 - val_loss: 0.4791 - val_accuracy: 0.7825
Epoch 2/100
2902/2902 [==============================] - 5s 2ms/step - loss: 0.6009 - accuracy: 0.7406 - val_loss: 0.7057 - val_accuracy: 0.7438
Epoch 3/100
2902/2902 [==============================] - 4s 2ms/step - loss: 0.5721 - accuracy: 0.7543 - val_loss: 0.7136 - val_accuracy: 0.6622
Epoch 4/100
2902/2902 [==============================] - 4s 1ms/step - loss: 0.5527 - accuracy: 0.7605 - val_loss: 0.5219 - val_accuracy: 0.7487
Epoch 4: early stopping
227/227 [==============================] - 0s 1ms/step - loss: 0.5219 - accuracy: 0.7487
Epoch 1/100
2902/2902 [==============================] - 6s 2ms/step - loss: 1.0667 - accuracy: 0.7177 - val_loss: 0.9848 - val_accuracy: 0.6841
Epoch 2/100
2902/2902 [==============================] - 4s 1ms/step - loss: 0.5502 - accuracy: 0.7490 - val_loss: 0.5305 - val_accuracy: 0.7457
Epoch 3/100
2902/